In [18]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [19]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, Input, LSTM
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

In [20]:
# Charger le modèle
from keras.models import load_model
model = load_model('/content/drive/My Drive/projet/Global_mod_SEA_LSTM_FED_ADV.h5')

# Charger les nouvelles données
import pandas as pd
new_df = pd.read_csv("/content/drive/My Drive/projet/dataset_APT.csv")

# Effectuer les mêmes étapes de prétraitement que pour les données d'entraînement
new_df1 = new_df.drop(['Flow ID', 'Src IP', 'Dst IP', 'Timestamp','Activity'], axis=1)
new_df1['Stage'] = new_df1['Stage'].replace('BENIGN', 'Benign')
new_df1 = new_df1[new_df1['Stage'] != 'Data Exfiltration']
categorical_cols = ['Protocol','Stage']
label_encoder = LabelEncoder()
new_df1[categorical_cols] = new_df1[categorical_cols].apply(label_encoder.fit_transform)
null_counts = new_df1.isnull().sum()
columns_with_null = null_counts[null_counts > 0].index
new_df1.dropna(inplace=True)
new_df1["Timestamp"] = pd.to_datetime(new_df["Timestamp"], format="%d/%m/%Y %I:%M:%S %p", dayfirst=True)
new_df1 = new_df1.sort_values(by='Timestamp')
target = new_df1["Stage"]
numeric_columns = new_df1.select_dtypes(include=['float64', 'int64', 'int32']).columns
correlation_matrix = new_df1[numeric_columns].corr()
correlation_with_target = correlation_matrix['Stage'].abs().sort_values(ascending=False)
threshold = 0.1
relevant_columns = correlation_with_target[correlation_with_target >= threshold].index
selected_data = new_df1[relevant_columns.union(['Stage'])]
new_df1 = selected_data
numerical_cols = new_df1.columns.drop('Stage')
scaler = StandardScaler()
new_df1[numerical_cols] = scaler.fit_transform(new_df1[numerical_cols])
Y_new = new_df1['Stage']
X_new = new_df1[new_df1.columns.drop('Stage')]
X_new = X_new.to_numpy()
Y_new = Y_new.to_numpy()

def create_sequences(data, labels, time_steps):
    X_seq = []
    Y_seq = []
    for i in range(len(data) - time_steps + 1):
        X_seq.append(data[i : i + time_steps])
        Y_seq.append(labels[i + time_steps - 1])
    return np.array(X_seq), np.array(Y_seq)

# Créer les séquences temporelles pour les nouvelles données
time_steps = 20  # Utiliser la même valeur que pour les données d'entraînement
X_new_seq, Y_new_seq = create_sequences(X_new, Y_new, time_steps)

# Prédire les classes pour les nouvelles séquences
y_pred_prob = model.predict(X_new_seq)  # Obtenir les probabilités de chaque classe
y_pred_class = np.argmax(y_pred_prob, axis=1)  # Obtenir la classe avec la probabilité la plus élevée

# Afficher les résultats
print("Les classes prédites sont:", y_pred_class)

# Mapping des classes prédites à des étiquettes significatives
class_labels = ["Benign", "Attack"]

# Assurer que les valeurs prédites sont dans la plage des indices de class_labels
y_pred_class = np.clip(y_pred_class, 0, len(class_labels) - 1)

y_pred_labels = [class_labels[pred_class] for pred_class in y_pred_class]

# Afficher les résultats
for i, label in enumerate(y_pred_labels):
    print(f"Séquence {i + 1}: Prédiction = {label}")




Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Séquence 81658: Prédiction = Benign
Séquence 81659: Prédiction = Benign
Séquence 81660: Prédiction = Benign
Séquence 81661: Prédiction = Benign
Séquence 81662: Prédiction = Benign
Séquence 81663: Prédiction = Benign
Séquence 81664: Prédiction = Benign
Séquence 81665: Prédiction = Benign
Séquence 81666: Prédiction = Benign
Séquence 81667: Prédiction = Benign
Séquence 81668: Prédiction = Benign
Séquence 81669: Prédiction = Benign
Séquence 81670: Prédiction = Benign
Séquence 81671: Prédiction = Benign
Séquence 81672: Prédiction = Benign
Séquence 81673: Prédiction = Benign
Séquence 81674: Prédiction = Benign
Séquence 81675: Prédiction = Benign
Séquence 81676: Prédiction = Benign
Séquence 81677: Prédiction = Benign
Séquence 81678: Prédiction = Benign
Séquence 81679: Prédiction = Benign
Séquence 81680: Prédiction = Benign
Séquence 81681: Prédiction = Benign
Séquence 81682: Prédiction = Benign
Séquence 81683: Prédic

In [ ]:
# Enregistrer les résultats dans un fichier texte
result_file_path = "/content/drive/My Drive/projet/resultats.txt"

with open(result_file_path, "w") as result_file:
    result_file.write("Séquence\tPrédiction\n")
    for i, label in enumerate(y_pred_labels):
        result_file.write(f"{i + 1}\t{label}\n")

print("Résultats enregistrés dans le fichier:", result_file_path)

In [ ]:
# Compter le nombre de paquets normaux et d'attaques
num_benign = np.sum(y_pred_class == class_labels.index("Benign"))
num_attack = np.sum(y_pred_class == class_labels.index("Attack"))

print("Nombre de paquets normaux :", num_benign)
print("Nombre de paquets d'attaque :", num_attack)